In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
# needed whenever working with spark dataframes
from pyspark.sql import *
!rm -rf metastore_db/
sqlContext = SQLContext(sc)

In [3]:
!wget https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv

--2018-02-16 08:58:14--  https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv [following]
--2018-02-16 08:58:15--  https://ibm.ent.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/n3opYxQlIERpLpMatA1qUOSW8bke2u2lNhg4k4GpVI8__TaKozknycmCkNUW7YV_aM7Hhvh4HZ1zpNMR6vSC8bKs1HHQnqjgXZXJ_1OjVB12zVrvlMvtt9nM9KpFOBHzCidlnM3BxMhm8pW1FWMaq9eddStXF1BwVcz8ix71mB3eW27OuaT3yDgcl7kYoQxBJ0FjGJv7w8O7NpfkNvZK0cxjq2B2-Y_vRoHzUk66tPAJyDj7-ArkPXalh3IuN-EnMm2YFdXVgZPRZH5qXl6uf9VYL

In [17]:
# using inferschema is very handy, but adds a second pass over the data

df = sqlContext.read.format('com.databricks.spark.csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .option('mode', 'DROPMALFORMED')\
                .load('f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv')

In [5]:
df.count()

32

In [6]:
df.printSchema()

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: integer (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: integer (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: integer (nullable = true)
 |-- am: integer (nullable = true)
 |-- gear: integer (nullable = true)
 |-- carb: integer (nullable = true)



In [7]:
df.show(5)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [8]:
df.select('car', 'mpg').show(5)

+-----------------+----+
|              car| mpg|
+-----------------+----+
|        Mazda RX4|21.0|
|    Mazda RX4 Wag|21.0|
|       Datsun 710|22.8|
|   Hornet 4 Drive|21.4|
|Hornet Sportabout|18.7|
+-----------------+----+
only showing top 5 rows



In [9]:
df.filter(df['mpg'] < 18).show(5)

+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
|        car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
| Duster 360|14.3|  8|360.0|245|3.21|3.57|15.84|  0|  0|   3|   4|
|  Merc 280C|17.8|  6|167.6|123|3.92|3.44| 18.9|  1|  0|   4|   4|
| Merc 450SE|16.4|  8|275.8|180|3.07|4.07| 17.4|  0|  0|   3|   3|
| Merc 450SL|17.3|  8|275.8|180|3.07|3.73| 17.6|  0|  0|   3|   3|
|Merc 450SLC|15.2|  8|275.8|180|3.07|3.78| 18.0|  0|  0|   3|   3|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
only showing top 5 rows



In [10]:
# the filter can be just like a SQL where clause
df.filter('mpg < 18').show(5)

+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
|        car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
| Duster 360|14.3|  8|360.0|245|3.21|3.57|15.84|  0|  0|   3|   4|
|  Merc 280C|17.8|  6|167.6|123|3.92|3.44| 18.9|  1|  0|   4|   4|
| Merc 450SE|16.4|  8|275.8|180|3.07|4.07| 17.4|  0|  0|   3|   3|
| Merc 450SL|17.3|  8|275.8|180|3.07|3.73| 17.6|  0|  0|   3|   3|
|Merc 450SLC|15.2|  8|275.8|180|3.07|3.78| 18.0|  0|  0|   3|   3|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
only showing top 5 rows



In [19]:
# dataframes are immutable -- withColumn creates a new one
df = df.withColumn('wtTon', df['wt'] * 0.45)
df.show(6)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-------+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|  wtTon|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-------+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|  1.179|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|1.29375|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|  1.044|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|1.44675|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|  1.548|
|          Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|  1.557|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-------+
only showing top 6 rows



In [20]:
df.groupby(['cyl'])\
.agg({"wt": "AVG"})\
.show(5)

+---+------------------+
|cyl|           avg(wt)|
+---+------------------+
|  6| 3.117142857142857|
|  4| 2.285727272727273|
|  8|3.9992142857142867|
+---+------------------+



In [21]:
df.groupby(['cyl'])\
    .agg({'wtTon': 'AVG'})\
    .show(5)

+---+------------------+
|cyl|        avg(wtTon)|
+---+------------------+
|  6|1.4027142857142856|
|  4|1.0285772727272726|
|  8|1.7996464285714289|
+---+------------------+



In [22]:
car_counts = df.groupby(['cyl'])\
        .agg({"*": "count"})\
        .sort("count(1)", ascending=False)\
        .show(5)

+---+--------+
|cyl|count(1)|
+---+--------+
|  8|      14|
|  4|      11|
|  6|       7|
+---+--------+



In [29]:
df.createOrReplaceTempView("cars")

# SQL statements can be run by using the sql method
highgearcars = sqlContext.sql("SELECT car, cyl,gear FROM cars WHERE cyl >= 4 AND cyl <= 9")
highgearcars.sort('cyl', ascending=True).show(12)

+--------------+---+----+
|           car|cyl|gear|
+--------------+---+----+
|     Merc 240D|  4|   4|
|      Fiat 128|  4|   4|
|   Honda Civic|  4|   4|
|    Datsun 710|  4|   4|
| Toyota Corona|  4|   3|
|     Fiat X1-9|  4|   4|
| Porsche 914-2|  4|   5|
|  Lotus Europa|  4|   5|
|    Volvo 142E|  4|   4|
|Toyota Corolla|  4|   4|
|      Merc 230|  4|   4|
|     Mazda RX4|  6|   4|
+--------------+---+----+
only showing top 12 rows



## Exercise 1:

Step 1: Create a list of tuples ("john", 23), ("mr. bean", 56), ("bill clinton", 70)

Step 2: Convert the list to a data frame with two columns

Step 3: Filter the people with age < 40

Step 4: Get the names ONLY of people smaller than 40

Step 5: Register the dataframe as a SQL table

Step 6: Select the names only of people whose age < 40

Some hints: http://spark.apache.org/docs/latest/sql-programming-guide.html#generic-loadsave-functions

In [34]:
names = [('john', 23), ('mr. bean', 56), ('bill clinton', 70)]
df = sqlContext.createDataFrame([(value[0], value[1]) for value in names], ['name', 'age'])
df.show()

+------------+---+
|        name|age|
+------------+---+
|        john| 23|
|    mr. bean| 56|
|bill clinton| 70|
+------------+---+



In [35]:
df.filter('age < 40').show()

+----+---+
|name|age|
+----+---+
|john| 23|
+----+---+



In [38]:
df.filter('age < 40').select('name').show()

+----+
|name|
+----+
|john|
+----+



In [39]:
df.createOrReplaceTempView('people')

In [41]:
sqlContext.sql('SELECT name FROM people WHERE age < 40').show()

+----+
|name|
+----+
|john|
+----+



## Exercise 2:

Step 1: Create a list of tuple ("john", "male"), ("hilary", "female")

Step 2: Join with the data from the previos exercise and output all males older than 50

Hints: https://databricks.com/blog/2015/02/17/introducing-dataframes-in-spark-for-large-scale-data-science.html

In [43]:
names_2 = [('john', 'male'), ('hilary', 'female')]
df_2 = sqlContext.createDataFrame([(value[0] ,value[1]) for value in names_2], ['name', 'sex'])

In [44]:
df_3 = df.join(df_2, on='name', how='outer')
df_3.show()

+------------+----+------+
|        name| age|   sex|
+------------+----+------+
|      hilary|null|female|
|bill clinton|  70|  null|
|    mr. bean|  56|  null|
|        john|  23|  male|
+------------+----+------+



In [47]:
df_3.filter('(sex == "male") and (age > 50)').show()

+----+---+---+
|name|age|sex|
+----+---+---+
+----+---+---+



In [ ]:
sc.stop()